In [1]:
%pip install -U langgraph langchain_community langchain_anthropic langchain_experimental

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install python-dotenv langchain_groq langchain_ollama


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:

import getpass
import os
from dotenv import find_dotenv, load_dotenv
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama

dotenv_path = find_dotenv()
load_dotenv(dotenv_path)
groq_api_key = os.getenv("groq_api_key")
os.environ["groq_api_key"] = groq_api_key 

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("groq_api_key")

dotenv_path = find_dotenv()
load_dotenv(dotenv_path)
tavily_api_key = os.getenv("tavily_api_key")
os.environ["tavily_api_key"] = tavily_api_key

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("tavily_api_key")

In [4]:
from typing import Annotated

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL

tavily_tool = TavilySearchResults(max_results=5)

# This executes code locally, which can be unsafe
repl = PythonREPL()


@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code and do math. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n\`\`\`python\n{code}\n\`\`\`\nStdout: {result}"
    return result_str

@tool
def custom_string_tool() -> str:
    """Returns a specific string when called."""
    return "This is the specific string you requested."


@tool
def custom_summary_report_tool() -> str:
    """Returns a summary report"""
    return "This is the summary report. 2023: Total Food - 99200, Total Transpo - 30900, Total Daily - 18750; 2024: Total Food - 104800, Total Transpo - 32750, Total Daily - 21100"


<>:23: SyntaxWarning: invalid escape sequence '\`'
<>:23: SyntaxWarning: invalid escape sequence '\`'
<>:23: SyntaxWarning: invalid escape sequence '\`'
<>:23: SyntaxWarning: invalid escape sequence '\`'
C:\Users\paulo\AppData\Local\Temp\ipykernel_7296\484249122.py:23: SyntaxWarning: invalid escape sequence '\`'
  result_str = f"Successfully executed:\n\`\`\`python\n{code}\n\`\`\`\nStdout: {result}"
C:\Users\paulo\AppData\Local\Temp\ipykernel_7296\484249122.py:23: SyntaxWarning: invalid escape sequence '\`'
  result_str = f"Successfully executed:\n\`\`\`python\n{code}\n\`\`\`\nStdout: {result}"


In [5]:
from typing import Literal
from typing_extensions import TypedDict

from langchain_anthropic import ChatAnthropic
from langgraph.graph import MessagesState, END
from langgraph.types import Command


members = ["researcher", "coder", "expenses_manager"]
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = members + ["FINISH"]

system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    f" following workers: {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)


class Router(TypedDict):
    """Worker to route to next. If no workers needed, route to FINISH."""

    next: Literal[*options]


#llm = ChatAnthropic(model="claude-3-5-sonnet-latest")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama-3.3-70b-Versatile")
llm_tool=ChatGroq(groq_api_key=groq_api_key,model_name="Llama-3.3-70b-Versatile")
#MODEL_NAME = "deepseek-r1:latest"

#MODEL_NAME = "gemma3:4b"
MODEL_NAME = "llama3.2:latest"
#llm = ChatOllama(model=MODEL_NAME, temperature=0.5)

MODEL_NAME = "llama3.2:latest"
#llm_tool = ChatOllama(model=MODEL_NAME, temperature=0.5)
class State(MessagesState):
    next: str

    
call_count = 0
call_count_max = 2
def supervisor_node(state: State) -> Command[Literal[*members, "__end__"]]:
    global call_count  # Now it refers to the global variable
    global call_count_max

    call_count = call_count +1

    messages = [
        {"role": "system", "content": system_prompt},
    ] + state["messages"]
    response = llm.with_structured_output(Router).invoke(messages)
    goto = response["next"]
    if goto == "FINISH":
        goto = END

    if call_count  == call_count_max:
        goto = END

    return Command(goto=goto, update={"next": goto})

In [6]:
result = llm.invoke("state")

In [7]:
result

AIMessage(content='The word "state" can have multiple meanings depending on the context. Here are a few possible interpretations:\n\n1. **Government or country**: A state can refer to a sovereign entity with its own government, laws, and international recognition. Examples include the United States, China, or France.\n2. **Condition or status**: A state can describe a particular condition or status of something, such as a state of mind (e.g., happy, sad, or anxious), a state of being (e.g., awake, asleep, or alive), or a state of affairs (e.g., good, bad, or uncertain).\n3. **Region or territory**: In the United States, a state can refer to one of the 50 constituent entities that make up the country, such as California, New York, or Texas.\n4. **Science and physics**: In physics, a state can refer to a specific configuration or condition of a system, such as a state of matter (e.g., solid, liquid, or gas) or a quantum state (e.g., a specific energy level or spin orientation).\n5. **Com

In [8]:
result

AIMessage(content='The word "state" can have multiple meanings depending on the context. Here are a few possible interpretations:\n\n1. **Government or country**: A state can refer to a sovereign entity with its own government, laws, and international recognition. Examples include the United States, China, or France.\n2. **Condition or status**: A state can describe a particular condition or status of something, such as a state of mind (e.g., happy, sad, or anxious), a state of being (e.g., awake, asleep, or alive), or a state of affairs (e.g., good, bad, or uncertain).\n3. **Region or territory**: In the United States, a state can refer to one of the 50 constituent entities that make up the country, such as California, New York, or Texas.\n4. **Science and physics**: In physics, a state can refer to a specific configuration or condition of a system, such as a state of matter (e.g., solid, liquid, or gas) or a quantum state (e.g., a specific energy level or spin orientation).\n5. **Com

In [9]:
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent


research_agent = create_react_agent(
    llm_tool, tools=[tavily_tool], prompt="You are a researcher. DO NOT do any math."
)


def research_node(state: State) -> Command[Literal["supervisor"]]:
    #print("state:",state)
    result = research_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="researcher")
            ]
        },
        goto="supervisor",
    )


# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION, WHICH CAN BE UNSAFE WHEN NOT SANDBOXED
code_agent = create_react_agent(llm, tools=[python_repl_tool])


def code_node(state: State) -> Command[Literal["supervisor"]]:
    #print("state:",state)
    result = code_agent.invoke(state)

    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="coder")
            ]
        },
        goto="supervisor",
    )


import json
# Load the JSON file
file_path = "monthly_expenses.json"  # Replace with the correct path to your file
with open(file_path, "r") as file:
    data = json.load(file)

# Convert the JSON data to a string
monthly_expenses_json_string = json.dumps(data, indent=4)  # Pretty-printed string

expenses_manager_agent = create_react_agent(llm, tools=[custom_summary_report_tool],prompt=f"You are a expenses manager. you have access to a tool that gives the sumary of the report if the quesiton is not there use this data{monthly_expenses_json_string}")

def expenses_manager_node(state: State) -> Command[Literal["supervisor"]]:

    result = expenses_manager_agent.invoke(state)

    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="expenses_manager")
            ]
         
        },
        goto="supervisor",
    )


builder = StateGraph(State)
builder.add_edge(START, "supervisor")
builder.add_node("supervisor", supervisor_node)
builder.add_node("researcher", research_node)
builder.add_node("coder", code_node)
builder.add_node("expenses_manager", expenses_manager_node)
graph = builder.compile()

In [15]:
from IPython.display import display, Image

#display(Image(graph.get_graph().draw_mermaid_png()))

In [10]:
def multi_agent_bot(prompt):
    global call_count
    global call_count_max

    call_count = 0
    call_count_max = 2
    events = graph.stream(
        {"messages": [("user", prompt)]}, subgraphs=True
    )

    my_array = []
    for s in events:
        print(s)
        my_array.append(s)

    agents = []
    j = -2
    for i in range(len(my_array)):
        
        try:
            if my_array[i][1]['supervisor']['next']:  # Check if 'next' key exists and is not empty
                

                if (j == -2):
                    agent =  my_array[i][1]['supervisor']['next']
                    print("AGENT:", agent)


                j = j+1
            
                agent =  my_array[i][1]['supervisor']['next']
            
                agents.append(agent)

                #agent = agent.append(my_array[i][1]['supervisor']['next'])
                print(my_array[i - 1][1][agents[j]]["messages"][0].content)
                print("-----------")

                print("AGENT:", agent)
                
                
                
        except (KeyError, IndexError, TypeError):
            pass

In [11]:
prompt = "give me a summary of the report of the expenses"
multi_agent_bot(prompt)


((), {'supervisor': {'next': 'expenses_manager'}})
(('expenses_manager:6e544f65-cd4b-5eca-9b12-002b038b1a21',), {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_khy0', 'function': {'arguments': '{}', 'name': 'custom_summary_report_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 1029, 'total_tokens': 1040, 'completion_time': 0.04, 'prompt_time': 0.106521043, 'queue_time': 0.242548645, 'total_time': 0.146521043}, 'model_name': 'Llama-3.3-70b-Versatile', 'system_fingerprint': 'fp_72a5dc99ee', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d72f0b61-16a5-4014-bd3a-2d455708fda2-0', tool_calls=[{'name': 'custom_summary_report_tool', 'args': {}, 'id': 'call_khy0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1029, 'output_tokens': 11, 'total_tokens': 1040})]}})
(('expenses_manager:6e544f65-cd4b-5eca-9b12-002b038b1a21',), {'tools': {'messages': [ToolMessage(content='This 

In [12]:
data 

{'2023': {'January': {'Food': 8000, 'Transpo': 2500, 'Daily': 1500},
  'February': {'Food': 7500, 'Transpo': 2300, 'Daily': 1400},
  'March': {'Food': 8500, 'Transpo': 2600, 'Daily': 1600},
  'April': {'Food': 8000, 'Transpo': 2400, 'Daily': 1500},
  'May': {'Food': 7800, 'Transpo': 2500, 'Daily': 1450},
  'June': {'Food': 8200, 'Transpo': 2600, 'Daily': 1500},
  'July': {'Food': 8500, 'Transpo': 2700, 'Daily': 1550},
  'August': {'Food': 8700, 'Transpo': 2800, 'Daily': 1600},
  'September': {'Food': 8100, 'Transpo': 2400, 'Daily': 1500},
  'October': {'Food': 8300, 'Transpo': 2500, 'Daily': 1550},
  'November': {'Food': 8600, 'Transpo': 2600, 'Daily': 1600},
  'December': {'Food': 9000, 'Transpo': 3000, 'Daily': 2000}},
 '2024': {'January': {'Food': 8500, 'Transpo': 2600, 'Daily': 1700},
  'February': {'Food': 8000, 'Transpo': 2500, 'Daily': 1600},
  'March': {'Food': 8700, 'Transpo': 2700, 'Daily': 1750},
  'April': {'Food': 8500, 'Transpo': 2600, 'Daily': 1700},
  'May': {'Food': 84

In [13]:
def calculate_totals(data):
    yearly_totals = {}
    for year, months in data.items():
        food_total = sum(month["Food"] for month in months.values())
        transpo_total = sum(month["Transpo"] for month in months.values())
        daily_total = sum(month["Daily"] for month in months.values())
        
        yearly_totals[year] = {
            "Total Food": food_total,
            "Total Transpo": transpo_total,
            "Total Daily": daily_total
        }
    return yearly_totals

result = calculate_totals(data)
print(json.dumps(result, indent=4))


{
    "2023": {
        "Total Food": 99200,
        "Total Transpo": 30900,
        "Total Daily": 18750
    },
    "2024": {
        "Total Food": 104800,
        "Total Transpo": 32750,
        "Total Daily": 21100
    }
}
